In [1]:
import os

out = 'bert-large-bahasa-standard-cased'
os.makedirs(out, exist_ok=True)

In [2]:
directory = out

In [7]:
from transformers import BertTokenizer, BertModel, BertConfig, AutoTokenizer, AutoModelWithLMHead, pipeline

In [6]:
import transformers
transformers.__version__

'3.0.2'

In [4]:
tokenizer = BertTokenizer('BERT.wordpiece', do_lower_case = False)
tokenizer.save_pretrained('bert-large-bahasa-standard-cased')

('bert-large-bahasa-standard-cased/vocab.txt',
 'bert-large-bahasa-standard-cased/special_tokens_map.json',
 'bert-large-bahasa-standard-cased/added_tokens.json')

In [5]:
tokenizer = BertTokenizer.from_pretrained('./bert-large-bahasa-standard-cased', do_lower_case = False)

In [9]:
# !transformers-cli convert --model_type bert \
#   --tf_checkpoint bert-base/model.ckpt-500000 \
#   --config BASE_config.json \
#   --pytorch_dump_output bert-base-bahasa-standard-cased/pytorch_model.bin

In [10]:
config = BertConfig(f'{directory}/config.json')
config.vocab_size = 32000

In [11]:
model = AutoModelWithLMHead.from_pretrained('./bert-base-bahasa-standard-cased/pytorch_model.bin', config = config)

/home/husein/.local/lib/python3.6/site-packages/transformers/modeling_auto.py:798: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,
Some weights of the model checkpoint at ./bert-base-bahasa-standard-cased/pytorch_model.bin were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a B

In [13]:
fill_mask = pipeline('fill-mask', model=model, tokenizer=tokenizer)

In [14]:
fill_mask('makan ayam dengan [MASK]')

[{'sequence': '[CLS] makan ayam dengan ayam [SEP]',
  'score': 0.16134978830814362,
  'token': 5838,
  'token_str': 'ayam'},
 {'sequence': '[CLS] makan ayam dengan itik [SEP]',
  'score': 0.054859865456819534,
  'token': 27174,
  'token_str': 'itik'},
 {'sequence': '[CLS] makan ayam dengan telur [SEP]',
  'score': 0.028192562982439995,
  'token': 8900,
  'token_str': 'telur'},
 {'sequence': '[CLS] makan ayam dengan sos [SEP]',
  'score': 0.025005826726555824,
  'token': 18840,
  'token_str': 'sos'},
 {'sequence': '[CLS] makan ayam dengan sambal [SEP]',
  'score': 0.02045358717441559,
  'token': 24098,
  'token_str': 'sambal'}]

In [15]:
fill_mask('mahathir sebenarnya sangat [MASK] tanah airnya')

[{'sequence': '[CLS] mahathir sebenarnya sangat mencintai tanah airnya [SEP]',
  'score': 0.09542842209339142,
  'token': 14727,
  'token_str': 'mencintai'},
 {'sequence': '[CLS] mahathir sebenarnya sangat cintakan tanah airnya [SEP]',
  'score': 0.06102743372321129,
  'token': 24174,
  'token_str': 'cintakan'},
 {'sequence': '[CLS] mahathir sebenarnya sangat suka tanah airnya [SEP]',
  'score': 0.05743848904967308,
  'token': 3085,
  'token_str': 'suka'},
 {'sequence': '[CLS] mahathir sebenarnya sangat sayangkan tanah airnya [SEP]',
  'score': 0.04153375327587128,
  'token': 22562,
  'token_str': 'sayangkan'},
 {'sequence': '[CLS] mahathir sebenarnya sangat luas tanah airnya [SEP]',
  'score': 0.033663298934698105,
  'token': 6425,
  'token_str': 'luas'}]

In [16]:
model.save_pretrained('bert-base-bahasa-standard-cased')

In [9]:
!transformers-cli upload ./bert-base-bahasa-standard-cased